## Importing library and requesting data from https://dev.socrata.com/foundry/data.cityofnewyork.us/k397-673e

In [2]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

%pip install pandas
%pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cityofnewyork.us,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 10000000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("k397-673e", limit=1000000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Python38\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Python38\python.exe -m pip install --upgrade pip' command.


## Part I: Data Exploration

In [12]:
# Checking the type for each of the columns
# We will find that all of the columns seem to be objects so we need to convert some to numeric later on or just drop
# them if not interested in them

results_df.dtypes

fiscal_year                   object
payroll_number                object
agency_name                   object
last_name                     object
first_name                    object
mid_init                      object
agency_start_date             object
work_location_borough         object
title_description             object
leave_status_as_of_july_31    object
base_salary                   object
pay_basis                     object
regular_hours                 object
regular_gross_paid            object
ot_hours                      object
total_ot_paid                 object
total_other_pay               object
total_gross_pay               object
dtype: object

In [ ]:
# Convert the leave_status_as_of_july_31 feature as 0 or 1


In [3]:
# Lets examine how much data we have for each of the fiscal year

unique_years = results_df['fiscal_year'].unique()
unique_years

for i in unique_years:
    print(i, ' - Number of Rows: ', results_df[results_df['fiscal_year'] == i].shape[0])

2021  - Number of Rows:  130750
2020  - Number of Rows:  128406
2019  - Number of Rows:  131457
2018  - Number of Rows:  124622
2017  - Number of Rows:  142445
2016  - Number of Rows:  116483
2015  - Number of Rows:  115347
2014  - Number of Rows:  110490


In [6]:
# Lets see how much missing data there is
# There is 17 rows that is missing the title description, 
# work_location_borough and agency_start_date which is a feature we are interested in
# Other features like payroll_number, name, mid_init we aren't interested in for this data exploration

results_df.isna().sum()



fiscal_year                        0
payroll_number                375210
agency_name                        0
last_name                        257
first_name                       259
mid_init                      479098
agency_start_date                 63
work_location_borough         109555
title_description                 17
leave_status_as_of_july_31         0
base_salary                        0
pay_basis                          0
regular_hours                      0
regular_gross_paid                 0
ot_hours                           0
total_ot_paid                      0
total_other_pay                    0
dtype: int64

#### Feature Engineering
We are mostly interested in knowing total gross pay of these jobs: Regular Gross Paid + Total OT Paid + Total Other Pay,
Therefore we will make a calculated field called total_gross_pay

We also need to convert these data types to numeric since python has read them in as strings.

We can also potentially drop base_salary and pay_basis since those are implicitely included in regular_gross_paid

In [18]:
results_df.drop(columns=['base_salary', 'pay_basis'])

# We will also drop some rows that have mistyped 60157.150.004245.78 as the regular_gross_paid
# Using regular expression to remove strings that have more than one decimal point in them

reg_pattern = "(\.[a-z]){2,}"
results_df = results_df[results_df['total_gross_pay'] != "60157.150.004245.78"]

convert_to_numeric = ['regular_hours', 'regular_gross_paid', 'ot_hours', 'total_ot_paid', 'total_other_pay', 'total_gross_pay']

for i in convert_to_numeric:
    results_df[i] = pd.to_numeric(results_df[i]) 

C:\Users\salim\AppData\Local\Temp\ipykernel_4300\2655663079.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df[i] = pd.to_numeric(results_df[i])


ValueError: Unable to parse string "53581.19650.48758.64" at position 0

In [8]:
# Creating a new column: total_gross_pay = regular_gross_pay + total_OT_paid + Total_Other_Pay

results_df['total_gross_pay'] = results_df['regular_gross_paid'] + results_df['total_ot_paid'] + results_df['total_other_pay']

In [11]:
# Creating a new column: ot_paid_per_hour = total_ot_paid / ot_hours

results_df['ot_paid_per_hour'] = results_df['total_ot_paid'] / results_df['ot_hours'] 

TypeError: unsupported operand type(s) for /: 'str' and 'str'

We can use construct a covariance matrix to see the relationship between each of the numerical categories.

In [10]:
%pip install seaborn
%pip install matplotlib

import seaborn as sns
import matplotlib.pyplot as plt

corr = results_df.corr()
sns.heatmap(corr, annot=True)


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Python38\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Python38\python.exe -m pip install --upgrade pip' command.


ValueError: zero-size array to reduction operation fmin which has no identity